## simple async /await demo

In [1]:
# this is a normal function
def function():
    return 1
function()

1

In [2]:
## this is generator function
def generator():
    yield 1
list(generator())

[1]

In [3]:
## Asynchronous normal function
async def async_function():
    return 1
function()

1

In [4]:
## Asynchronous generator function
async def async_generator():
    yield 1
generator()

<generator object generator at 0x108f81518>

In [5]:
import types
print(type(function) is types.FunctionType)
print(type(generator()) is types.GeneratorType)
print(type(async_function()) is types.CoroutineType)
print(type(async_generator()) is types.AsyncGeneratorType)

True
True
True
True


/Volumes/Transcend/Dropbox/code/3env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: coroutine 'async_function' was never awaited
  after removing the cwd from sys.path.


## how to use Asynchronous normal function

Asynchronous normal function will trow a StopIteration exception with the function return values in it.


In [6]:
try:
    async_function().send(None)
except StopIteration as e:
    print(e.value)
# 1

1


In [7]:
def run(coroutine):
    try:
        coroutine.send(None)
    except StopIteration as e:
        return e.value

In [8]:
run(async_function())

1

in Asynchronous normal function we can use __await__ keyword hangup the this Asynchronous function and wait other Asynchronous function 

In [13]:
import time 
from datetime import datetime
async def async_function():
    time.sleep(2)
    return 1

async def await_coroutine():
    print("before get seconds {}".format(datetime.now().second))
    result = await async_function()
    print("after get seconds {}".format(datetime.now().second))
    print(result)

In [14]:
run(await_coroutine())

before get seconds 45
after get seconds 47
1


1. keyword __await__ must be  used inside of ___async def___
2. keyword __await__  must be followed by a Awaitable object
3. Awaitable object has \__await___ method

In [21]:
## the interface of awatiable object
from abc import ABCMeta,abstractmethod
class Awaitable(metaclass=ABCMeta):
    __slot__ = ()
    
    @abstractmethod
    def __await__(self):
        yield
    
    @classmethod
    def __subclass__(cls,C):
        if cls is Awaitable:
            return _check_methods(C,"__await__")
        return NotImplemented

Coroutine inherit from Awaitable and implements send,throw close methods

In [22]:
class Coroutine(Awaitable):
    __slots__ = ()

    @abstractmethod
    def send(self, value):
        ...

    @abstractmethod
    def throw(self, typ, val=None, tb=None):
        ...

    def close(self):
        ...
        
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Coroutine:
            return _check_methods(C, '__await__', 'send', 'throw', 'close')
        return NotImplemented  

## Asynchronous generator demo

In [24]:
class Potato:
    @classmethod
    def make(cls, num, *args, **kws):
        potatos = []
        for i in range(num):
            potatos.append(cls.__new__(cls, *args, **kws))
        return potatos

all_potatos = Potato.make(5)

basic code :

In [27]:
# this code will goes into deadlock,because you will never get the potatos
def take_potatos(num):
    count = 0
    while True:
        if len(all_potatos) == 0:
            sleep(.1)
        else:
            potato = all_potatos.pop()
            yield potato
            count += 1
            if count == num:
                break

def buy_potatos():
    bucket = []
    for p in take_potatos(50):
        bucket.append(p)

try async

In [31]:
async def take_potatos(num):
    count = 0
    while count < num:
        if len(all_potatos) == 0:
            await make_more_potatos()
        else:
            potato = all_potatos.pop()
            yield potato
            count += 1
            
async def buy_potatos():
    bucket = []
    async for p in take_potatos(50):
        bucket.append(p)
        print(f'Got potato {id(p)}...')

In [32]:
async def make_more_potatos():
#     all_potatos = Potato.make(5)
    yield time.sleep(1)
    all_potatos.extend(Potato.make(random.randint(1, 10)))

In [40]:
def main():
    import asyncio
    loop = asyncio.get_event_loop()
    res = loop.run_until_complete(asyncio.wait([buy_potatos()]))
    loop.close()

In [44]:
def sub_gen():
    yield 1
    yield 2
    yield 3

def gen():
    return (yield from sub_gen())

def main():
    for val in gen():
        print(val)
# 1
# 2
# 3

In [46]:
main()

1
2
3


In [48]:
# https://docs.python.org/3.4/library/asyncio-task.html
import asyncio

@asyncio.coroutine
def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    yield from asyncio.sleep(1.0)
    return x + y

@asyncio.coroutine
def print_sum(x, y):
    result = yield from compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
loop.close()

RuntimeError: This event loop is already running

Compute 1 + 2 ...
1 + 2 = 3


In [49]:
import asyncio
import types

@types.coroutine
def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    yield from asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
loop.close()

RuntimeError: This event loop is already running

Compute 1 + 2 ...
1 + 2 = 3
